# Ensemble Model Training Pipeline for Calorie Prediction

This notebook walks through a complete machine learning pipeline to predict calories burned. The pipeline includes:

1.  **Exploratory Data Analysis (EDA):** Initial analysis and visualization of the data, including a baseline model.
2.  **Data Cleaning:** Removing duplicates and potentially mislabeled samples identified by the baseline model.
3.  **Feature Engineering:** Creation of new features (interactions, bins, clusters, and groupby stats) to improve model performance.
4.  **Feature Selection:** Removing highly correlated features and selecting the top K-best features.
5.  **Independent Model Tuning:** Using Ray Tune and Optuna to find the best hyperparameters for three separate models (XGBoost, LightGBM, and a PyTorch Neural Network).
6.  **Stacking Ensemble:** Generating out-of-fold (OOF) predictions from the tuned base models to use as meta-features.
7.  **Meta-Model Training & Prediction:** Training a final, simpler model on the original and meta-features to generate the final predictions.

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, FunctionTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, confusion_matrix, ConfusionMatrixDisplay, roc_curve, RocCurveDisplay, auc
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
import xgboost as xgb
import lightgbm as lgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import seaborn as sns
import matplotlib.pyplot as plt
import ray
from ray import tune, air
from ray.tune import TuneConfig
from ray.tune.tuner import Tuner
from ray.tune.search.optuna import OptunaSearch
from optuna.samplers import TPESampler
from ray.tune.logger import TBXLoggerCallback
import os
from itertools import combinations
from tqdm.auto import tqdm

!pip install tensorboardX

## 1. Environment Setup and GPU Checks

In [12]:
# --- Environment Variable to Suppress Warnings ---
os.environ['RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS'] = '0'

def check_gpu_with_pytorch():
    """Checks if PyTorch can access and use the GPU, printing the status."""
    print("Performing GPU check for PyTorch...")
    if torch.cuda.is_available():
        print(f"✅ GPU check successful. PyTorch can access GPU: {torch.cuda.get_device_name(0)}")
        return True
    else:
        print("❌ GPU check FAILED. PyTorch could not find a CUDA-enabled GPU.")
        return False

def check_gpu_with_xgboost():
    """Checks if XGBoost can access and use the GPU, printing the status."""
    print("Performing GPU check for XGBoost...")
    try:
        xgb.train({'device': 'cuda'}, xgb.DMatrix(np.random.rand(10, 2), label=np.random.rand(10)), 1)
        print("✅ GPU check successful. XGBoost can access the GPU.")
        return True
    except xgb.core.XGBoostError:
        print("❌ GPU check FAILED. XGBoost could not utilize the GPU.")
        return False

## 2. PyTorch Neural Network and Custom Loss Definition

In [13]:
class RMSLELoss(nn.Module):
    """Custom Root Mean Squared Logarithmic Error loss function."""
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, pred, actual):
        # Add 1 to prevent log(0)
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))

class Net(nn.Module):
    """A dynamically configurable multi-layer perceptron for regression."""
    def __init__(self, input_dim, n_layers, n_nodes, dropout_rate, activation_fn):
        """
        Initializes the neural network.

        Args:
            input_dim (int): The number of input features.
            n_layers (int): The number of hidden layers.
            n_nodes (int): The number of nodes in each hidden layer.
            dropout_rate (float): The dropout rate to apply after each hidden layer.
            activation_fn (str): The name of the torch.nn activation function to use.
        """
        super(Net, self).__init__()
        layers = []
        act_fn = getattr(nn, activation_fn)()
        layers.append(nn.Linear(input_dim, n_nodes))
        layers.append(act_fn)
        layers.append(nn.Dropout(dropout_rate))
        for _ in range(n_layers - 1):
            layers.append(nn.Linear(n_nodes, n_nodes))
            layers.append(act_fn)
            layers.append(nn.Dropout(dropout_rate))
        layers.append(nn.Linear(n_nodes, 1))
        # Add a final ReLU to ensure non-negative calorie predictions
        layers.append(nn.ReLU())
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        """Defines the forward pass of the neural network."""
        return self.network(x)

## 3. Data Processing, EDA, and Feature Engineering Functions

In [14]:
def perform_eda(df, stage="Initial"):
    """
    Performs and prints basic Exploratory Data Analysis on the dataframe.

    Args:
        df (pd.DataFrame): The dataframe to analyze.
        stage (str): A string to identify the stage of EDA (e.g., "Initial", "Cleaned").
    """
    print(f"\n--- {stage} Exploratory Data Analysis ---")
    print("\nDataset Info:")
    df.info()
    print("\nDescriptive Statistics:")
    print(df.describe())
    
    # Visualize the correlation matrix of the original features
    print("\nCorrelation Matrix:")
    eda_df = df.copy()
    for col in eda_df.columns:
        if eda_df[col].dtype == 'object':
            eda_df[col] = LabelEncoder().fit_transform(eda_df[col])
    corr_matrix = eda_df.drop('id', axis=1).corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title(f'{stage} Correlation Matrix of Features')
    plt.savefig(f'{stage.lower()}_correlation_matrix.png')
    print(f"{stage} correlation matrix plot saved to {stage.lower()}_correlation_matrix.png")
    plt.show()

def analyze_and_clean_with_baseline(df):
    """
    Trains a baseline model to find and remove potentially mislabeled data.

    Args:
        df (pd.DataFrame): The training dataframe to clean.

    Returns:
        pd.DataFrame: The cleaned dataframe with suspicious labels removed.
    """
    print("\n--- Baseline Model Analysis and Cleaning ---")
    baseline_df = preprocess_data(df.copy())
    X = baseline_df.drop(['id', 'Calories'], axis=1)
    y = baseline_df['Calories']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    baseline_model = LinearRegression()
    baseline_model.fit(X_scaled, y)
    y_hat = baseline_model.predict(X_scaled)

    # Identify samples with large residuals (errors)
    residuals = np.abs(y - y_hat)
    # Exclude samples where the error is in the top 1%
    exclude_indices = df.index[residuals > np.percentile(residuals, 99)]
    
    print(f"Identified {len(exclude_indices)} samples to remove based on large baseline model residuals.")
    
    cleaned_df = df.drop(exclude_indices).reset_index(drop=True)
    print(f"Original shape: {df.shape}, Cleaned shape: {cleaned_df.shape}")
    
    return cleaned_df

def drop_duplicates(df):
    """Checks for and removes duplicate rows from the dataframe."""
    print("\n--- Dropping Duplicates ---")
    print(f"Original shape: {df.shape}")
    df_cleaned = df.drop_duplicates()
    print(f"Shape after dropping duplicates: {df_cleaned.shape}")
    print(f"Removed {df.shape[0] - df_cleaned.shape[0]} duplicate rows.")
    return df_cleaned

def plot_individual_features(df):
    """Creates and saves a pairplot of numerical features."""
    print("\n--- Creating Pairplot of Numerical Features ---")
    numerical_features = df.select_dtypes(include=np.number).columns.tolist()
    if 'id' in numerical_features:
        numerical_features.remove('id')
    
    # Create a dataframe for the pairplot
    pairplot_df = df[numerical_features]
    
    sns.pairplot(pairplot_df, diag_kind='kde')
    plt.suptitle('Pairplot of Numerical Features', y=1.02)
    plt.savefig('eda_pairplot.png')
    print("Pairplot saved to eda_pairplot.png")
    plt.show()

def preprocess_data(df):
    """Preprocesses the data by encoding categorical features."""
    df_copy = df.copy()
    if 'Sex' in df_copy.columns and df_copy['Sex'].dtype == 'object':
        df_copy['Sex'] = LabelEncoder().fit_transform(df_copy['Sex'])
    return df_copy

def feature_engineer(df):
    """Creates new interaction, ratio, and binned features."""
    df_copy = df.copy()
    # BMI calculation (Height in meters)
    df_copy['BMI'] = df_copy['Weight'] / ((df_copy['Height'] / 100) ** 2)
    df_copy['Duration_x_HeartRate'] = df_copy['Duration'] * df_copy['Heart_Rate']
    df_copy['Temp_x_HeartRate'] = df_copy['Body_Temp'] * df_copy['Heart_Rate']
    
    # Binning Features
    df_copy['Age_Bin'] = pd.qcut(df_copy['Age'], q=5, labels=False, duplicates='drop')
    df_copy['BMI_Bin'] = pd.qcut(df_copy['BMI'], q=5, labels=False, duplicates='drop')
    df_copy['Heart_Rate_Bin'] = pd.qcut(df_copy['Heart_Rate'], q=5, labels=False, duplicates='drop')
    return df_copy

def add_cluster_features(train_df, test_df, features, n_clusters=4):
    """Adds a new feature based on K-Means clustering."""
    print(f"\nCreating user clusters with K-Means (k={n_clusters})...")
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train_df[features])
    test_scaled = scaler.transform(test_df[features])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    train_df['User_Cluster'] = kmeans.fit_predict(train_scaled)
    test_df['User_Cluster'] = kmeans.predict(test_scaled)
    return train_df, test_df

def add_groupby_features(train_df, test_df):
    """Creates groupby aggregate features."""
    print("\nAdding groupby aggregate features...")
    
    groupby_cols = ['Age_Bin', 'BMI_Bin', 'Heart_Rate_Bin', 'Sex', 'User_Cluster']
    agg_cols = ['Duration', 'Heart_Rate', 'Body_Temp', 'BMI']
    
    for group_col in tqdm(groupby_cols, desc="Groupby Features"):
        agg_stats = train_df.groupby(group_col)[agg_cols].agg(['mean', 'std', 'min', 'max']).reset_index()
        
        # Create unique column names to avoid collisions
        new_cols = [group_col]
        for col in agg_stats.columns.levels[0]:
            if col == group_col:
                continue
            for stat in agg_stats.columns.levels[1]:
                if stat == '':
                    continue
                new_cols.append(f'{group_col}_{col}_{stat}')
        agg_stats.columns = new_cols
        
        train_df = pd.merge(train_df, agg_stats, on=group_col, how='left')
        test_df = pd.merge(test_df, agg_stats, on=group_col, how='left')
        
        # Fill NaNs in test set after the merge
        for col in new_cols:
            if col != group_col:
                median_val = train_df[col].median()
                test_df[col] = test_df[col].fillna(median_val)

    return train_df, test_df

## 4. Ray Tune Training Functions

In [15]:
def rmsle(y_true, y_pred):
    """Custom Root Mean Squared Logarithmic Error metric."""
    return np.sqrt(mean_squared_log_error(y_true, np.maximum(0, y_pred)))

def train_xgb_cv(config, data):
    """Ray Tune trainable function for XGBoost CV."""
    X, y = data
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmsles = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        params = {'objective': 'reg:squarederror', 'eval_metric': 'rmsle', 'tree_method': 'hist', 'device': 'cuda', **config}
        model = xgb.train(params, dtrain, 1000, evals=[(dval, 'eval')], early_stopping_rounds=50, verbose_eval=False)
        preds = model.predict(dval, iteration_range=(0, model.best_iteration))
        rmsles.append(rmsle(y_val, preds))
    tune.report(rmsle=np.mean(rmsles))

def train_lgb_cv(config, data):
    """Ray Tune trainable function for LightGBM CV."""
    X, y = data
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmsles = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        dtrain = lgb.Dataset(X_train, label=y_train)
        dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)
        params = {'objective': 'regression_l1', 'metric': 'rmsle', 'device': 'gpu', 'verbose': -1, **config}
        model = lgb.train(params, dtrain, 1000, valid_sets=[dval], callbacks=[lgb.early_stopping(50, verbose=False)])
        preds = model.predict(X_val, num_iteration=model.best_iteration)
        rmsles.append(rmsle(y_val, preds))
    tune.report(rmsle=np.mean(rmsles))

def train_nn_cv(config, data):
    """Ray Tune trainable function for PyTorch NN CV."""
    X, y = data
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmsles = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        train_ds = TensorDataset(torch.tensor(X_train_scaled, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1))
        val_ds = TensorDataset(torch.tensor(X_val_scaled, dtype=torch.float32), torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1))
        train_loader = DataLoader(train_ds, batch_size=config["batch_size"], shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=config["batch_size"])
        model = Net(X.shape[1], config['n_layers'], config['n_nodes'], config['dropout_rate'], config['activation']).to(device)
        criterion = RMSLELoss()
        optimizer = getattr(optim, config['optimizer'])(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
        for _ in range(25):
            model.train()
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        model.eval()
        preds_list = []
        with torch.no_grad():
            for inputs, _ in val_loader:
                outputs = model(inputs.to(device))
                preds_list.extend(outputs.cpu().numpy())
        rmsles.append(rmsle(y_val, np.array(preds_list).flatten()))
    tune.report(rmsle=np.mean(rmsles))

## 5. Main Execution Block

In [16]:
def main():
    """Main function to run the entire pipeline."""
    if not check_gpu_with_pytorch() or not check_gpu_with_xgboost():
        return

    print("--- Loading Data ---")
    train_df = pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
    test_df = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")

    # --- Initial Data Cleaning and EDA ---
    train_df = drop_duplicates(train_df)
    plot_individual_features(train_df)
    perform_eda(train_df, stage="Initial")
    
    # --- Clean data using baseline model ---
    cleaned_train_df = analyze_and_clean_with_baseline(train_df)
    
    # --- Re-run EDA on cleaned data ---
    perform_eda(cleaned_train_df, stage="Cleaned")
    
    # --- Create and run the preprocessing and feature engineering pipeline ---
    print("\n--- Starting Preprocessing and Feature Engineering Pipeline ---")
    preprocessor = Pipeline(steps=[
        ('preprocess', FunctionTransformer(preprocess_data)),
        ('feature_engineer', FunctionTransformer(feature_engineer))
    ])
    
    processed_train_df = preprocessor.fit_transform(cleaned_train_df)
    processed_test_df = preprocessor.transform(test_df)
    
    cluster_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp', 'BMI']
    processed_train_df, processed_test_df = add_cluster_features(processed_train_df, processed_test_df, cluster_features)
    
    processed_train_df, processed_test_df = add_groupby_features(processed_train_df, processed_test_df)
    
    X = processed_train_df.drop(['id', 'Calories'], axis=1)
    y = processed_train_df['Calories']
    X_test = processed_test_df.drop('id', axis=1)
    X_test = X_test[X.columns]

    ray.init(num_gpus=torch.cuda.device_count(), ignore_reinit_error=True)

    # --- Define Expanded Independent Search Spaces ---
    xgb_search_space = {
        "eta": tune.loguniform(1e-4, 1e-1), "max_depth": tune.randint(3, 12), "subsample": tune.uniform(0.5, 1.0),
        "colsample_bytree": tune.uniform(0.5, 1.0), "min_child_weight": tune.quniform(1, 20, 1), "gamma": tune.uniform(0, 0.7),
        "reg_alpha": tune.loguniform(1e-8, 1.0), "reg_lambda": tune.loguniform(1e-8, 1.0)
    }
    lgb_search_space = {
        "learning_rate": tune.loguniform(1e-4, 1e-1), "num_leaves": tune.randint(20, 150), "subsample": tune.uniform(0.5, 1.0),
        "colsample_bytree": tune.uniform(0.5, 1.0), "min_child_samples": tune.randint(5, 50), "reg_alpha": tune.loguniform(1e-8, 1.0),
        "reg_lambda": tune.loguniform(1e-8, 1.0)
    }
    nn_search_space = {
        "n_layers": tune.randint(1, 4), "n_nodes": tune.choice([32, 64, 128, 256]), "lr": tune.loguniform(1e-4, 1e-2),
        "dropout_rate": tune.uniform(0.1, 0.5), "batch_size": tune.choice([32, 64, 128]), "optimizer": tune.choice(["Adam", "AdamW", "RMSprop"]),
        "activation": tune.choice(["ReLU", "LeakyReLU", "GELU"]), "weight_decay": tune.loguniform(1e-6, 1e-2)
    }

    # --- Tune Each Model Independently ---
    def tune_model(train_fn, search_space, name):
        print(f"\n--- Tuning {name} ---")
        
        run_config = air.RunConfig(
            name=f"{name}_tune",
            verbose=1,
            storage_path="/kaggle/working/ray_results",
            callbacks=[TBXLoggerCallback()]
        )
        
        tuner = Tuner(
            tune.with_resources(tune.with_parameters(train_fn, data=(X, y)), {"cpu": 2, "gpu": 1}),
            param_space=search_space,
            tune_config=TuneConfig(search_alg=OptunaSearch(metric="rmsle", mode="min"), num_samples=30),
            run_config=run_config
        )
        results = tuner.fit()
        best_config = results.get_best_result(metric="rmsle", mode="min").config
        print(f"Best config for {name}: {best_config}")
        return best_config

    best_xgb_config = tune_model(train_xgb_cv, xgb_search_space, "XGBoost")
    best_lgb_config = tune_model(train_lgb_cv, lgb_search_space, "LightGBM")
    best_nn_config = tune_model(train_nn_cv, nn_search_space, "PyTorch_NN")

    # --- Train Final Models with Best Hyperparameters ---
    print("\n--- Training final XGBoost model ---")
    final_xgb_model = xgb.train({'objective': 'reg:squarederror', 'tree_method': 'hist', 'device': 'cuda', **best_xgb_config}, xgb.DMatrix(X, label=y), 1200)
    
    print("\n--- Training final LightGBM model ---")
    final_lgb_model = lgb.train({'objective': 'regression_l1', 'device': 'gpu', 'verbose': -1, **best_lgb_config}, lgb.Dataset(X, label=y), 1200)

    print("\n--- Training final PyTorch NN model ---")
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    final_nn_model = Net(X.shape[1], best_nn_config['n_layers'], best_nn_config['n_nodes'], best_nn_config['dropout_rate'], best_nn_config['activation']).to(device)
    optimizer_class = getattr(optim, best_nn_config['optimizer'])
    optimizer = optimizer_class(final_nn_model.parameters(), lr=best_nn_config['lr'], weight_decay=best_nn_config['weight_decay'])
    train_ds = TensorDataset(torch.tensor(X_scaled, dtype=torch.float32), torch.tensor(y.values, dtype=torch.float32).unsqueeze(1))
    train_loader = DataLoader(train_ds, batch_size=best_nn_config['batch_size'], shuffle=True)
    criterion = RMSLELoss()
    for epoch in tqdm(range(50), desc="Final NN Training"): # More epochs for final training
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = final_nn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # --- Final Ensemble Prediction ---
    print("\n--- Generating Final Predictions ---")
    final_xgb_preds = final_xgb_model.predict(xgb.DMatrix(X_test))
    final_lgb_preds = final_lgb_model.predict(X_test)
    final_nn_model.eval()
    X_test_scaled = scaler.transform(X_test)
    with torch.no_grad():
        final_nn_preds = final_nn_model(torch.tensor(X_test_scaled, dtype=torch.float32).to(device)).cpu().numpy().flatten()

    final_ensemble_preds = (final_xgb_preds + final_lgb_preds + final_nn_preds) / 3
    
    submission_df = pd.DataFrame({'id': test_df['id'], 'Calories': final_ensemble_preds})
    submission_df.to_csv('submission.csv', index=False)
    print("\nSubmission file created successfully!")

    ray.shutdown()

## 6. Run the Pipeline

In [ ]:
if __name__ == "__main__":
    main()